This code provides a structured process for reading a CSV file, manually reviewing rows to determine if a response was written by a human or a model, and calculating accuracy based on user input:

1. **`read_csv(file_path)`**: Reads a CSV file and prompts the user to select a range of rows to work with. The function returns the selected rows, along with the starting and ending row indices.

2. **`save_results(results, first_row, last_row, file_path)`**: Saves the results of user evaluations to a new CSV file, named based on the original file and the selected row range.

3. **`ask_user_and_record(data)`**: Iterates through the selected rows, presenting the user with content and asking them to classify it as written by a human or a model. The user can exit early by pressing 'E'.

4. **`calculate_accuracy(user_answers, correct_answers)`**: Computes the accuracy of the user's classifications by comparing them to the correct labels.



In [ ]:
import pandas as pd
import re
from google.colab import files

In [ ]:
def read_csv(file_path):
    try:
        df = pd.read_csv(file_path)
        num_rows = len(df)

        if num_rows == 0:
            print("The file is empty.")
            return None, None, None

        while True:
            try:
                first_row = int(input(f"Enter the first row (0 to {num_rows - 1}): "))
                if 0 <= first_row < num_rows:
                    break
                else:
                    print(f"Please enter a valid row number between 0 and {num_rows - 1}.")
            except ValueError:
                print("Invalid input. Please enter an integer.")

        while True:
            try:
                last_row = int(input(f"Enter the last row ({first_row} to {num_rows - 1}): "))
                if first_row <= last_row < num_rows:
                    break
                else:
                    print(f"Please enter a valid row number between {first_row} and {num_rows - 1}.")
            except ValueError:
                print("Invalid input. Please enter an integer.")

        return df.iloc[first_row:last_row + 1], first_row, last_row

    except Exception as e:
        print(f"Error reading file {file_path}: {e}")
        return None, None, None


In [ ]:
def save_results(results, first_row, last_row, file_path):
    try:
        filename = f"{file_path.rsplit('.', 1)[0]}_{first_row}_{last_row}.csv"
        results.to_csv(filename, index=False)
        print(f"Results saved to {filename}")
    except Exception as e:
        print(f"Error saving file {file_path}: {e}")

In [ ]:
def ask_user_and_record(data):
    user_answers = []
    correct_answers = []
    total_questions = len(data)
    question_count = 0
    last_processed_row = None

    for idx, row in data.iterrows():
        question_count += 1
        cleaned_reply_text = re.sub(r'\s+', ' ', row['reply_text']).strip()
        print(f"----------------------------------------{question_count}/{total_questions}-----------------------------------------------\n")
        print(
            f"Subreddit: {row['subreddit_name']}\nConversation name: {row['conversation_title']}\n\nComment:{row['top_level_text']}\n\nReply: {cleaned_reply_text}")
        user_input = None
        while user_input not in ['0', '1', 'E']:
            user_input = input("\nWas this written by a human (0) or a model (1)? (Press 'E' to exit early): ").strip()
            if user_input not in ['0', '1', 'E']:
                print("Invalid input. Please enter 0 for Human, 1 for Model, or 'E' to exit.")

        if user_input == 'E':
            print("Exiting early.")
            break

        user_answers.append(int(user_input))
        correct_answers.append(row['labels'])
        last_processed_row = idx  # Record the last processed row index

    return user_answers, correct_answers, last_processed_row


In [ ]:
def calculate_accuracy(user_answers, correct_answers):
    correct_count = sum([1 for user_ans, correct_ans in zip(user_answers, correct_answers) if user_ans == correct_ans])
    return correct_count / len(correct_answers)

In [ ]:
# Upload the test_datasets from Data/DataSets/Small Dataset/CSV/test_dataset.csv
uploaded = files.upload()


This code orchestrates the process of evaluating a dataset by prompting the user to manually classify whether each response was written by a human or a model. Here's a breakdown:

1. **File Handling**: The code specifies input (`test_by_post_shuffeled.csv`) and output (`results.csv`) CSV files. It reads the input data and allows the user to select a subset of rows for evaluation.

2. **User Interaction**: The `ask_user_and_record` function is used to present each selected row to the user, asking them to classify it. The user's answers are recorded along with the correct labels.

3. **Accuracy Calculation**: After the user finishes classifying (or exits early), the code calculates the accuracy of the user's classifications.

4. **Saving Results**: The evaluated subset, including the original data, user predictions, and correct labels, is saved to a new CSV file named `results.csv`.



In [ ]:
input_csv = "test_by_post_shuffeled.csv"
output_csv = "results.csv"
data, first_row, last_row = read_csv(input_csv)

if data is not None:
    user_answers, correct_answers, last_processed_row = ask_user_and_record(data)

    if last_processed_row is not None:
        accuracy = calculate_accuracy(user_answers, correct_answers)
        print(f"Your accuracy: {accuracy * 100:.2f}%")

        results = data.iloc[:len(user_answers)].copy()  # Only include rows that were processed
        results['prediction'] = user_answers
        results['label'] = correct_answers

        results = results[['subreddit_name', 'conversation_title', 'top_level_text', 'reply_text', 'labels', 'prediction']]
        save_results(results, first_row, last_processed_row, output_csv)
    else:
        print("No questions were answered.")